In [ ]:
import pickle
import pandas as pd
import seaborn as sns
import sys
import shutil
from tqdm import tqdm

import bluepyopt as bpopt
import bluepyopt.ephys as ephys
import neuroplotlib as nplt

import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.spatial import distance
import MEAutility as mu
import json
import time
import numpy as np
from pathlib import Path
from pprint import pprint

import multimodalfitting as mf

%matplotlib notebook

In [ ]:
base_dir = Path("../../")
path_result = "./runs.pkl"

abd = False
cell_name = "cell1_211006_3148"
path_training = "all_responses_cell1.pkl"

# cell_name = "cell1_211011_3436"
# path_training = "all_responses_cell2.pkl"

path_testing = f"responses_testing_{cell_name}_{abd}.pkl"

model_name = cell_name
ephys_dir = base_dir / "experimental_data" / cell_name / "patch_data"

cell_models_folder = base_dir / "cell_models"
model_folder = cell_models_folder / cell_name

In [ ]:
colors_dict = {"soma": "C0",
               "all": "C1",
               "sections": "C2",
               "single": "C3"}
feature_sets = {"soma": "soma",
                "all": "extra",
                "sections": "extra",
                "single": "extra"}
figsize = (10, 7)

In [ ]:
data = pickle.load(open(path_result, 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == '{model_name}'")

strategies = np.unique(df_model.strategy)

opt_results_training = pickle.load(open(f"responses_training_{model_name}_{abd}.pkl", "rb"))

opt_results_test = None
if Path(path_testing).is_file():
    with open(path_testing, 'rb') as f:
        opt_results_test = pickle.load(f)
    compute_responses = False
else:
    compute_responses = True

In [ ]:
extra_kwargs = mf.utils.get_extra_kwargs()
extra_kwargs["ms_cut"] = [2, 5]

protocol_for_eap = "IDrest_300"

eva_extra_training = mf.create_evaluator(
    model_name=model_name,
    strategy="all",
    protocols_with_lfp=protocol_for_eap,
    abd=abd,
    **extra_kwargs
)

print(f"All test features --> num features {len(eva_extra_training.fitness_calculator.objectives)}")

In [ ]:
protocol_for_eap = "firepattern_200"
protocols_to_exclude = ["IV", "APWaveform", "IDrest"]

eva_extra_testing = mf.create_evaluator(
    model_name=model_name,
    strategy="all",
    protocols_with_lfp=protocol_for_eap,
    all_protocols=True,
    exclude_protocols=protocols_to_exclude,
    **extra_kwargs
)

print(f"All test features --> num features {len(eva_extra_testing.fitness_calculator.objectives)}")

In [ ]:
model_folder = Path("../../") / "cell_models" / cell_name
probe_file = model_folder / "fitting" / "efeatures" / "probe_BPO.json"

probe = mf.define_electrode(probe_file=probe_file)

morphology = model_folder / "morphology_corrected.swc"
cell = mf.create_experimental_model(model_name=model_name,
                                    morphology_file=morphology,
                                    cell_model_folder=model_folder,
                                    release=True,
                                    v_init=-80, 
                                    abd=False)
sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe)#, mechanisms_directory=model_folder)

In [ ]:
selected_strategies = ["soma", "all"]

## Load experimental responses

In [ ]:
from bluepyefe.extract import read_recordings, extract_efeatures_at_targets, compute_rheobase,\
    group_efeatures, create_feature_protocol_files, convert_legacy_targets
from bluepyefe.plotting import plot_all_recordings_efeatures

from multimodalfitting.efeatures_extraction import build_wcp_metadata, wcp_reader, get_ecode_targets, \
    ecodes_wcp_timings

In [ ]:
# define timings for this experiment
ecodes_cell_timings = {
    "IDthres": {
        'ton': 400,
        'toff': 670
    },
    "firepattern": {
        'ton': 500,
        'toff': 4100
    },
    "IV": {
        'ton': 400,
        'toff': 3400
    },
    "IDrest": {
        'ton': 400,
        'toff': 1750
    },
    "APWaveform": {
        'ton': 350,
        'toff': 400
    },
    "HyperDepol": {
        'ton': 400,
        'toff': 1120,
        'tmid': 850
    },
    "sAHP": {
        'ton': 400,
        'toff': 1325,
        'tmid': 650,
        'tmid2': 875
    },
    "PosCheops": {
        'ton': 1000,
        't1': 9000,
        't2': 10500,
        't3': 14500,
        't4': 16000,
        'toff': 18660
    }
}

In [ ]:
if cell_name == "cell1_211006_3148":
    runs = [1, 2, 3, 4, 5]  # run1 --> different rheobase
elif cell_name == "cell1_211011_3436":
    runs = [3, 4, 5, 6]
    
ecode_names = list(ecodes_cell_timings.keys())

files_list = []

for run in runs:
    rep_dict = {}
    for ecode in ecode_names:
        for patch_file in ephys_dir.iterdir():
            if f"run{run}" in patch_file.name and ecode.lower() in patch_file.name:
                rep_dict[ecode] = patch_file
    files_list.append(rep_dict)

In [ ]:
files_metadata = build_wcp_metadata(cell_id=cell_name, 
                                    files_list=files_list, 
                                    ecode_timings=ecodes_cell_timings, 
                                    repetition_as_different_cells=False)

cells = read_recordings(
    files_metadata=files_metadata,
    recording_reader=wcp_reader
)

# define target features for different protocols
targets = get_ecode_targets(ecodes_cell_timings)

global_tolerance = 30
for target in targets:
    target["tolerance"] = global_tolerance
    
if cell_name == "cell1_211006_3148":
    majority = 0.4
else:
    majority = 0.2
    
compute_rheobase(
    cells, 
    protocols_rheobase=['IDthres'],
    rheobase_strategy="majority",
    rheobase_settings={"majority": majority}
)

protocols = group_efeatures(cells, targets, use_global_rheobase=True)

In [ ]:
protocols_opt = list(eva_extra_testing.fitness_protocols.keys())

responses_experimental = []
responses_all = []
num_runs = len(protocols[0].recordings)
for run in range(num_runs):
    print(f"Populating responses for run {run}")
    response_dict = {}
    response_all_dict = {}
    for protocol in protocols:
        for i, rec in enumerate(protocol.recordings):
            if i == run:
                response = bpopt.ephys.responses.TimeVoltageResponse(name=protocol.name,
                                                                     time=rec.t, voltage=rec.voltage)
                response_all_dict[f"{protocol.stimulus_name}.soma.v"] = response
                if protocol.stimulus_name in protocols_opt:
                    response_dict[f"{protocol.stimulus_name}.soma.v"] = response
    responses_experimental.append(response_dict)
    responses_all.append(response_all_dict)

# Compute and plot best responses (testing data)

In [ ]:
if compute_responses:

    opt_results_test = {}

    for strategy in selected_strategies:

        print(f"Simulating best '{strategy}' -- seed: {opt_results_training[strategy]['best_seed']}")
        t_start = time.time()
        responses = eva_extra_testing.run_protocols(
            eva_extra_testing.fitness_protocols.values(), 
            param_values=opt_results_training[strategy]["best_params"]
        )
        print(f"Simulated responses in {np.round(time.time() - t_start, 2)} s")

        eap = mf.utils.calculate_eap(
            responses=responses, protocols=eva_extra_testing.fitness_protocols,
            protocol_name=protocol_for_eap, align_extra=True, **extra_kwargs)

        opt_results_test[strategy] = {
            "responses": responses,
            "eap": eap,
            "features": eva_extra_testing.fitness_calculator.calculate_scores(responses)
        }

    pickle.dump(opt_results_test, open(f"responses_testing_{model_name}_{abd}.pkl", "wb"))

In [ ]:
figs_intra = {}
figs_extra = {}

In [ ]:
print(opt_results_test["soma"]["responses"])
protocols_to_plot = ["firepattern_200", "sAHP_300", "PosCheops_300"]

for strategy in selected_strategies:
    responses_to_plot = [responses_experimental[0], opt_results_test[strategy]["responses"]]
    colors = ["k", colors_dict[strategy]]
    fig_intra = mf.plot_multiple_responses(
        responses_to_plot, 
        protocol_names=protocols_to_plot,
        colors=colors,
        titles=protocols_to_plot,
        return_fig=True, 
        labels=None)
    
    fig_intra.savefig(f"{strategy}_{cell_name}_responses_intra.pdf", transparent=True, bbox_inches="tight")

## Plot scores distributions (training data)

In [ ]:
import pandas

opt_results_df = pandas.read_pickle(path_training).query("abd == False").query("strategy == ['soma', 'all']")

order = ["soma", "all"]

colors = [colors_dict[strat] for strat in order]
fig_intra_seeds, ax = plt.subplots(figsize=(4.5, 10.5))
sns.boxplot(data=opt_results_df, x="strategy", y="intra_score", 
            order=order, showfliers=False,  ax=ax, palette=colors)
ax.set_xlabel("Strategy", fontsize=22)
ax.set_ylabel("Score", fontsize=22)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.tick_params(axis='both', which='major', labelsize=18)
ax.set_title("Intracellular\n(10 seeds)", fontsize=30)
fig_intra_seeds.savefig(f"{strategy}_{cell_name}_intra.pdf", transparent=True, bbox_inches="tight")

fig_extra_seeds, ax = plt.subplots(figsize=(4, 10))
sns.boxplot(data=opt_results_df, x="strategy", y="extra_score",
            showfliers=True,  order=order, palette=colors)
ax.set_xlabel("Strategy", fontsize=22)
ax.set_ylabel("Score", fontsize=22)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.tick_params(axis='both', which='major', labelsize=18)
ax.set_title("Extracellular\n(10 seeds)", fontsize=30)
fig_extra_seeds.savefig(f"{strategy}_{cell_name}_extra.pdf", transparent=True, bbox_inches="tight")

## Plot MEA responses

In [ ]:
eap_exp = np.load(model_folder / "fitting" / "efeatures" / "template_BPO.npy") / 1000
vscale = np.max(np.abs(eap_exp))

fig_extra, ax_extra = plt.subplots(figsize=figsize)

ax = mf.plot_cell(
    eva_extra_training.cell_model, sim,
    color="k",
    color_ais="g",
    color_dend="r",
    detailed=False,
    param_values=opt_results_training["all"]["best_params"],
    ax=ax_extra, 
    #exclude_sections=["soma"],
    alpha=0.3,
    lw=0.5
)

ax_extra = mu.plot_mea_recording(eap_exp, probe, vscale=vscale, lw=0.4, ax=ax_extra)
ax_extra = mu.plot_mea_recording(opt_results_test["soma"]["eap"], probe, ax=ax_extra, vscale=vscale, 
                                 colors=colors_dict["soma"], lw=0.4)
ax_extra = mu.plot_mea_recording(opt_results_test["all"]["eap"], probe, ax=ax_extra, vscale=vscale, 
                                 colors=colors_dict["all"], lw=0.4)
ax_extra.get_lines()[-1].set_label(strategy.upper())

fig_extra.savefig(f"{strategy}_{cell_name}_responses_extra.pdf", transparent=True, bbox_inches="tight")

In [ ]:
eap_exp = np.load(model_folder / "fitting" / "efeatures" / "template_BPO.npy") / 1000
vscale = np.max(np.abs(eap_exp))

fig_extra, ax_extra = plt.subplots(figsize=figsize)

ax = mf.plot_cell(
    eva_extra_training.cell_model, sim,
    color="k",
    color_ais="g",
    color_dend="r",
    detailed=False,
    param_values=opt_results_training["all"]["best_params"],
    ax=ax_extra, 
    #exclude_sections=["soma"],
    alpha=0.3,
    lw=0.5
)

ax_extra = mu.plot_mea_recording(eap_exp, probe, vscale=vscale, lw=0.4, ax=ax_extra)
ax_extra = mu.plot_mea_recording(opt_results_test["soma"]["eap"], probe, ax=ax_extra, vscale=vscale, 
                                 colors=colors_dict["soma"], lw=0.4)
ax_extra = mu.plot_mea_recording(opt_results_test["all"]["eap"], probe, ax=ax_extra, vscale=vscale, 
                                 colors=colors_dict["all"], lw=0.4)
ax_extra.get_lines()[-1].set_label(strategy.upper())

if cell_name == "cell1_211006_3148":
    ax_extra.set_xlim(-34, 98)
    ax_extra.set_ylim(-77, 22)
else:
    ax_extra.set_xbound(lower=-98, upper=42)
    ax_extra.set_ybound(lower=-47, upper=57)

fig_extra.savefig(f"{strategy}_{cell_name}_responses_extra_zoom.pdf", transparent=True, bbox_inches="tight")